## 1. Cleaning sequences using cutadapt

We will utilize the following code run in the terminal to clean the raw 16S rRNA sequences from marine sediments in fastq format (in the data/raw/16S_rRNA_seqs folder). First, you should navigate with cd to the folder that contains the raw sequences. Once you are there, run the following line to make a file with unique IDs for each sample: 

In [ ]:
ls DO-*R1* | cut -f 1,2 -d "_" > unique-sample-IDs.txt

The first command finds the files with the initials you want, in this case, "DO", and that include R1 (only forward sequences) in the filename. The second command extracts their identifiers by selecting only the first 2 "columns" before the first "_", and sends the output to the file unique-sample-IDs.txt. If your sequences have a different name formatting, you might need to modify this code to retrieve the correct IDs.

Now, we will remove the primer sequences using cutadapt. The sequences were amplified targeting the V4- V5 hypervariable region by PCR using universal primer set 515F (5' -GTGYCAGCMGCCGCGGTAA- 3′) and 926R (5′-CCGYCAATTYMTTTRAGTTT- 3′) (Caporaso et al., 2001).
- First, make sure you install cutadapt following the instructions here: https://cutadapt.readthedocs.io/en/stable/installation.html
- Second, you will use mkdir to create a separate directory for the trimmed reads:

In [ ]:
mkdir primer-trimmed-reads

In this case, you can find the trimmed reads folder inside in data/processed/16S_rRNA_seqs
- Then, you will execute the following command to verify that the looping through the sample IDs will be performed correctly by cutadapt - if not pointing to the correct location, we would get a “No such file or directory”-.

In [ ]:
for sample in $(cat unique-sample-IDs.txt)
do

    echo "On sample: ${sample}"

    echo "    Forward read file location:"
    ls ${sample}_L001_R1_001.fastq.gz
    
    echo "    Reverse read file location:"
    ls ${sample}_L001_R2_001.fastq.gz
    
    # just adding a blank line in between samples as they print out
    echo ""

done

If the unique-sample-IDs.txt file was created correctly, we should get a list of sample IDs with their corresponding filenames for the forward and reverse filenames. After this, you can run cutadapt to remove the primers from all the sequences (delete the comments after # before running):

In [ ]:
for sample in $(cat unique-sample-IDs.txt)
do

    echo "On sample: ${sample}"

    cutadapt -g GTGYCAGCMGCCGCGGTAA \ #this is the forward primer sequence to be found in the fwd read
             -G CCGYCAATTYMTTTRAGTTT \ #this is the reverse primer sequence to be found in the reverse read
             -o primer-trimmed-reads/${sample}-trimmed-R1.fastq.gz \ #this is the name that the output forward trimmed reads will have
             -p primer-trimmed-reads/${sample}-trimmed-R2.fastq.gz \ #this is the name that the output reverse trimmed reads will have
             --discard-untrimmed \ #untrimmed sequences will be discarded
             ${sample}_L001_R1_001.fastq.gz \
             ${sample}_L001_R2_001.fastq.gz \
             >> cutadapt-primer-trimming-output.txt 2>&1

done

To confirm that the output is correct, take a look at the generated file with the less command:

In [ ]:
less cutadapt-primer-trimming-output.txt

For this particular dataset, the Summary should read as follows:

In [ ]:
=== Summary ===

Total read pairs processed:             27,535
  Read 1 with adapter:                  27,006 (98.1%)
  Read 2 with adapter:                  27,265 (99.0%)
Pairs written (passing filters):        26,742 (97.1%)

Total basepairs processed:    16,576,070 bp
  Read 1:     8,288,035 bp
  Read 2:     8,288,035 bp
Total written (filtered):     15,055,874 bp (90.8%)
  Read 1:     7,541,306 bp
  Read 2:     7,514,568 bp

To exit the less output window, just press q. To identify the percentage of reads retained, we can select from the previous file only the lines that include "Pairs" with the grep command, and take a look at the first 3 lines of the file by running:

In [ ]:
grep "Pairs" cutadapt-primer-trimming-output.txt | head -n 3

To extract only the percentages, we can cut them out using the cut command and save them in a file called percent-reads-retained.tmp by running the following line:

In [ ]:
grep "Pairs" cutadapt-primer-trimming-output.txt | cut -f 3 -d "(" | tr -d ")" > percent-reads-retained.tmp

In this case, -f 3 extracts the third column, d specifices the delimiter as "(", and tr -d ")" removes the trailing ). Now, we will fuse this file with the uniqie-sample-IDs.txt file with paste into a new file called percent-reads-retained-after-primer-removal.tsv:

In [ ]:
paste unique-sample-IDs.txt percent-reads-retained.tmp\
> percent-reads-retained-after-primer-removal.tsv

We will remove the intermediate file with rm:

In [ ]:
rm percent-reads-retained.tmp

And finally, we will look at the table we created, that included only identifiers and the percentage of reads retained after primer removal using cat:

In [ ]:
cat percent-reads-retained-after-primer-removal.tsv